In [1]:
import numpy as np, pandas as pd, torch,os
from sentence_transformers import SentenceTransformer, util
import tensorflow_hub as hub
import tensorflow_text
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
ST_BERT = text_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

2023-05-23 10:22:13.638567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 10:22:13.824341: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 10:22:14.503494: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-23 10:22:14.503575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [2]:
import pickle

In [3]:
data_train = pd.read_csv("./dataset/pre-processing/IntraHD_Sentence_internal-train.csv")
# data_train['label'] = [1 if i=='Y' else 0 for i in data_train.label.values.tolist()]
data_valid = pd.read_csv("./dataset/pre-processing/IntraHD_Sentence_internal-test.csv")
# data_valid['label'] = [1 if i=='Y' else 0 for i in data_valid.label.values.tolist()]
print(data_train.shape, data_valid.shape)

(1011, 2) (113, 2)


In [4]:
print(len(data_train.label.values)-sum(data_train.label.values), sum(data_train.label.values))
print(len(data_valid.label.values)-sum(data_valid.label.values), sum(data_valid.label.values))

683 328
76 37


In [5]:
# pd.set_option('display.max_rows', 2000)
# pd.set_option('display.max_columns', 20)
# data_train[data_train.label==1]

In [8]:
save_path = "/ssd8/chih/project/yadong/predict_baseline_version02_ACM/dataset/IntraHD_pickle_save_05-17"
model_name_s = 'ST-DBMCv1'
save_type_folder = [f'{model_name_s}_sample1124_sequence_fullcontext']
for save_type_folder_ls in save_type_folder:
    for dp in ['','train','valid','test']:
        os.makedirs(os.path.join(save_path, save_type_folder_ls, dp), exist_ok=True)

In [9]:
BERT_training_embeddings=[]
run_data = data_train['處置其他+症狀處置（描述）'].astype(str).to_list()
BERT_training_embeddings = ST_BERT.encode(run_data)
save_folder = save_type_folder[0]
# save_folder = "USD_sample2693_contextsplit"
for idx, (embed_, text_, label_) in enumerate(zip(BERT_training_embeddings, data_train['處置其他+症狀處置（描述）'].to_list(),data_train['label'].to_list())):
    with open(os.path.join(save_path, save_folder, "train", "train_{}_internal_pk_data_{}_fixed_re-marked_merge.pkl".format(model_name_s, idx)), "wb") as f:
        save_dict = {"embed": embed_, "text": text_, "label": label_}
        pickle.dump(save_dict, f)

In [10]:
BERT_validation_embeddings=[]
run_data = data_valid['處置其他+症狀處置（描述）'].astype(str).to_list()
BERT_validation_embeddings = ST_BERT.encode(run_data)
save_folder = save_type_folder[0]
# save_folder = "USD_sample2693_contextsplit"
for idx, (embed_, text_, label_) in enumerate(zip(BERT_validation_embeddings, data_valid['處置其他+症狀處置（描述）'].to_list(),data_valid['label'].to_list())):
    with open(os.path.join(save_path, save_folder, "valid", "valid_{}_internal_pk_data_{}_fixed_re-marked_merge.pkl".format(model_name_s, idx)), "wb") as f:
        save_dict = {"embed": embed_, "text": text_, "label": label_}
        pickle.dump(save_dict, f)